偏心gyroidのための関数の準備

通常のジャイロイドは次の陰関数で表される.
$$ G(x,y,z) = \sin x \cos y + \sin y \cos z + \sin z \cos x = 0$$
ただし、$x,y,z$の定義域は$[0,2\pi]$ である。

これに対し、偏心gyroidは次の式となる。
$$ G(x,y,z,A) = \sin x \cos y + \sin y \cos z + \sin z \cos x + A = 0$$

`surface`関数はこれをそのまま実装する。

In [3]:
def surface(x, cell, A=0.0):
    Lx, Ly, Lz = cell[0, 0], cell[1, 1], cell[2, 2]
    assert Lx == Ly and Lx == Lz, "Should be a cubic cell."
    assert (
        np.count_nonzero(cell - np.diag(np.diagonal(cell))) == 0
    ), "Should be an orthogonal cell."

    rx, ry, rz = (2 * np.pi * x / L).T
    return (
        np.sin(rx) * np.cos(ry) + np.sin(ry) * np.cos(rz) + np.sin(rz) * np.cos(rx) + A
    )

これを$x,y,z$でそれぞれ微分してみる。

In [5]:
from sympy import sin, cos, diff
from sympy.abc import x, y, z, A, L, D, pi

rx = 2 * pi * x / L
ry = 2 * pi * y / L
rz = 2 * pi * z / L

G = sin(rx) * cos(ry) + sin(ry) * cos(rz) + sin(rz) * cos(rx) + A

diff(G, x)

-2*pi*sin(2*pi*x/L)*sin(2*pi*z/L)/L + 2*pi*cos(2*pi*x/L)*cos(2*pi*y/L)/L

これを`gradient`関数と呼ぶ。

In [ ]:
def gradient(x, cell):
    Lx, Ly, Lz = cell[0, 0], cell[1, 1], cell[2, 2]
    assert Lx == Ly and Lx == Lz, "Should be a cubic cell."
    assert (
        np.count_nonzero(cell - np.diag(np.diagonal(cell))) == 0
    ), "Should be an orthogonal cell."

    rx, ry, rz = (2 * np.pi * x / L).T
    return (
        (-np.sin(rx) * np.sin(rz) + np.cos(rx) * np.cos(ry)) * 2 * np.pi / L,
        (-np.sin(rx) * np.sin(ry) + np.cos(ry) * np.cos(rz)) * 2 * np.pi / L,
        (-np.sin(ry) * np.sin(rz) + np.cos(rx) * np.cos(rz)) * 2 * np.pi / L,
    )

コスト関数は、$D G^2$とする。これを最小にする$x,y,z$は$G$上にある。

構造最適化のためには、このコスト関数の勾配が必要。

In [6]:
diff(D*G**2, x)

D*(-4*pi*sin(2*pi*x/L)*sin(2*pi*z/L)/L + 4*pi*cos(2*pi*x/L)*cos(2*pi*y/L)/L)*(A + sin(2*pi*x/L)*cos(2*pi*y/L) + sin(2*pi*y/L)*cos(2*pi*z/L) + sin(2*pi*z/L)*cos(2*pi*x/L))

第1項は$dG/dx$, 第2項は$G$で、$1/2$や2は出現しないようだ。
